# Real-time Streaming ETL with Structured Streaming in Apache Spark 2.1
Part 1 of Scalable Data @ Databricks

- https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html
- https://databricks.com/blog/2016/07/28/structured-streaming-in-apache-spark.html
- https://databricks.com/blog/2016/07/28/continuous-applications-evolving-streaming-in-apache-spark-2-0.html
- http://cdn2.hubspot.net/hubfs/438089/notebooks/spark2.0/Structured%20Streaming%20using%20Python%20DataFrames%20API.html
- http://cdn2.hubspot.net/hubfs/438089/notebooks/spark2.0/Structured%20Streaming%20using%20Scala%20DataFrames%20API.html
- http://www.slideshare.net/databricks/a-deep-dive-into-structured-streaming
- https://www.rittmanmead.com/blog/2017/01/getting-started-with-spark-streaming-with-python-and-kafka/

- https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/8599738367597028/2070341989008532/3601578643761083/latest.html


In [23]:
import pixiedust

In [24]:
# The code was removed by DSX for sharing.

Batch/Interactive Processing
The usual first step in attempting to process the data is to interactively query the data. Let's define a static DataFrame on the files, and give it a table name.

There are about 50 JSON files in the directory. Let's see what each JSON file contains.
Each line in the file contains JSON record with two fields - time and action. Let's try to analyze these files interactively.

In [ ]:
from pyspark.sql.types import *

inputPath = 'swift://Databricks.' + name + '/file-*.json'

# Since we know the data format already, let's define the schema to speed up processing (no need for Spark to infer schema)
jsonSchema = StructType([ StructField("time", TimestampType(), True), StructField("action", StringType(), True) ])

# Static DataFrame representing data in the JSON files
staticInputDF = spark.read.schema(jsonSchema).json(inputPath)

display(staticInputDF)

Now we can compute the number of "open" and "close" actions with one hour windows. To do this, we will group by the action column and 1 hour windows over the time column.

In [37]:
from pyspark.sql.functions import *      # for window() function

staticCountsDF = (
  staticInputDF
    .groupBy(
       staticInputDF.action, 
       window(staticInputDF.time, "1 hour"))    
    .count()
)
staticCountsDF.cache()

# Register the DataFrame as table 'static_counts'
staticCountsDF.createOrReplaceTempView("static_counts")

Now we can directly use SQL to query the table. For example, here are the total counts across all the hours.

In [39]:
spark.sql("select * from static_counts").show(100, False)

+------+---------------------------------------------+-----+
|action|window                                       |count|
+------+---------------------------------------------+-----+
|Close |[2016-07-26 08:00:00.0,2016-07-26 09:00:00.0]|1028 |
|Open  |[2016-07-26 13:00:00.0,2016-07-26 14:00:00.0]|1004 |
|Close |[2016-07-26 21:00:00.0,2016-07-26 22:00:00.0]|971  |
|Open  |[2016-07-26 23:00:00.0,2016-07-27 00:00:00.0]|995  |
|Open  |[2016-07-27 00:00:00.0,2016-07-27 01:00:00.0]|986  |
|Open  |[2016-07-26 00:00:00.0,2016-07-26 01:00:00.0]|1000 |
|Open  |[2016-07-26 06:00:00.0,2016-07-26 07:00:00.0]|991  |
|Close |[2016-07-26 01:00:00.0,2016-07-26 02:00:00.0]|1011 |
|Close |[2016-07-27 00:00:00.0,2016-07-27 01:00:00.0]|987  |
|Open  |[2016-07-26 05:00:00.0,2016-07-26 06:00:00.0]|1007 |
|Close |[2016-07-25 23:00:00.0,2016-07-26 00:00:00.0]|815  |
|Close |[2016-07-26 11:00:00.0,2016-07-26 12:00:00.0]|984  |
|Close |[2016-07-26 22:00:00.0,2016-07-26 23:00:00.0]|1025 |
|Open  |[2016-07-27 02:0

In [40]:
spark.sql("select action, sum(count) as total_count from static_counts group by action").show()

+------+-----------+
|action|total_count|
+------+-----------+
|  Open|      50000|
| Close|      50000|
+------+-----------+



In [43]:
display(spark.sql('select action, date_format(window.end, "MMM-dd HH:mm") as time, count from static_counts order by time, action'))

action,time,count
Close,Jul-25 22:00,11
Open,Jul-25 22:00,179
Close,Jul-25 23:00,344
Open,Jul-25 23:00,1001
Close,Jul-26 00:00,815
Open,Jul-26 00:00,999
Close,Jul-26 01:00,1003
Open,Jul-26 01:00,1000
Close,Jul-26 02:00,1011
Open,Jul-26 02:00,993


Note the two ends of the graph. The close actions are generated such that they are after the corresponding open actions, so there are more "opens" in the beginning and more "closes" in the end.

# Stream Processing

Now that we have analyzed the data interactively, let's convert this to a streaming query that continuously updates as data comes. Since we just have a static set of files, we are going to emulate a stream from them by reading one file at a time, in the chronological order they were created. The query we have to write is pretty much the same as the interactive query above.

In [59]:
from pyspark.sql.functions import *

# Similar to definition of staticInputDF above, just using `readStream` instead of `read`
streamingInputDF = (
  spark
    .readStream                       
    .schema(jsonSchema)               # Set the schema of the JSON data
    .option("maxFilesPerTrigger", 1)  # Treat a sequence of files as a stream by picking one file at a time
    .json(inputPath)
)

# Same query as staticInputDF
streamingCountsDF = (                 
  streamingInputDF
    .groupBy(
      streamingInputDF.action, 
      window(streamingInputDF.time, "1 hour"))
    .count()
)

# Is this DF actually a streaming DF?
streamingCountsDF.isStreaming

True

As you can see, streamingCountsDF is a streaming Dataframe (streamingCountsDF.isStreaming was true). You can start streaming computation, by defining the sink and starting it. In our case, we want to interactively query the counts (same queries as above), so we will set the complete set of 1 hour counts to be a in a in-memory table (note that this for testing purpose only in Spark 2.0).

In [60]:
spark.conf.set("spark.sql.shuffle.partitions", "2")  # keep the size of shuffles small

query = (
  streamingCountsDF
    .writeStream
    .format("memory")        # memory = store in-memory table (for testing only in Spark 2.0)
    .queryName("counts")     # counts = name of the in-memory table
    .outputMode("complete")  # complete = all the counts should be in the table
    .start()
)

In [58]:
query.stop()

query is a handle to the streaming query that is running in the background. This query is continuously picking up files and updating the windowed counts.
Note the status of query in the above cell. Both the Status: ACTIVE and the progress bar shows that the query is active. Furthermore, if you expand the >Details above, you will find the number of files they have already processed.
Let's wait a bit for a few files to be processed and then interactively query the in-memory counts table.

In [46]:
from time import sleep
sleep(5)  # wait a bit for computation to start

In [49]:
display(spark.sql('select action, date_format(window.end, "MMM-dd HH:mm") as time, count from counts order by time, action'))

action,time,count
Close,Jul-25 22:00,11
Open,Jul-25 22:00,179
Close,Jul-25 23:00,344
Open,Jul-25 23:00,1001
Close,Jul-26 00:00,815
Open,Jul-26 00:00,999
Close,Jul-26 01:00,1003
Open,Jul-26 01:00,1000
Close,Jul-26 02:00,1011
Open,Jul-26 02:00,993


We see the timeline of windowed counts (similar to the static one ealrier) building up. If we keep running this interactive query repeatedly, we will see the latest updated counts which the streaming query is updating in the background.

In [50]:
sleep(5)

In [51]:
display(spark.sql('select action, date_format(window.end, "MMM-dd HH:mm") as time, count from counts order by time, action'))

action,time,count
Close,Jul-25 22:00,11
Open,Jul-25 22:00,179
Close,Jul-25 23:00,344
Open,Jul-25 23:00,1001
Close,Jul-26 00:00,815
Open,Jul-26 00:00,999
Close,Jul-26 01:00,1003
Open,Jul-26 01:00,1000
Close,Jul-26 02:00,1011
Open,Jul-26 02:00,993


In [52]:
sleep(5)

In [53]:
display(spark.sql('select action, date_format(window.end, "MMM-dd HH:mm") as time, count from counts order by time, action'))

action,time,count
Close,Jul-25 22:00,11
Open,Jul-25 22:00,179
Close,Jul-25 23:00,344
Open,Jul-25 23:00,1001
Close,Jul-26 00:00,815
Open,Jul-26 00:00,999
Close,Jul-26 01:00,1003
Open,Jul-26 01:00,1000
Close,Jul-26 02:00,1011
Open,Jul-26 02:00,993



Also, let's see the total number of "opens" and "closes".

In [65]:
display(spark.sql('select action, sum(count) as total_count from counts group by action order by action'))

action,total_count
Close,50000
Open,50000


If you keep running the above query repeatedly, you will always find that the number of "opens" is more than the number of "closes", as expected in a data stream where a "close" always appear after corresponding "open". This shows that Structured Streaming ensures prefix integrity. Read the blog posts linked below if you want to know more.
Note that there are only a few files, so consuming all of them there will be no updates to the counts. Rerun the query if you want to interact with the streaming query again.
Finally, you can stop the query running in the background, either by clicking on the 'Cancel' link in the cell of the query, or by executing query.stop(). Either way, when the query is stopped, the status of the corresponding cell above will automatically update to TERMINATED.

In [66]:
query.stop()